In [67]:
# Loading data
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import helper_functions

table_path = "diagrams and tables/"

table_1_7 = helper_functions.import_file(table_path+"/1_7.csv")
rohdaten = helper_functions.import_file("Rohdaten.csv")
rohdaten.dropna(inplace=True)

https://www.statstest.com/factorial-anova/

https://stats.oarc.ucla.edu/stata/dae/ordered-logistic-regression/

https://stats.oarc.ucla.edu/other/mult-pkg/whatstat/

https://www.researchgate.net/post/How-do-I-check-for-statistical-significance-between-3-groups-using-percentage-values-determined-from-categorical-data
-> use Chi Square pairwise


In [68]:
chi_test_data = rohdaten[["Chainsaw","Feller Buncher","Excavator","Region"]]

In [85]:
regions = chi_test_data["Region"].unique()
machines = ["Chainsaw","Feller Buncher","Excavator"]
machine_region_dict = {}

# Get the sums of each machine per region
for region in regions:
    sum_per_machine = chi_test_data.loc[chi_test_data["Region"]==region,machines].sum()
    #print(region," :",sum_per_machine)
    machine_region_dict[region] = sum_per_machine

In [72]:
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
import itertools

# Chi Square Test - pairwise comparison of each region, is there a significant difference between them?
machine_df = pd.DataFrame(index=regions,columns=regions)
for region_a, region_b in set(itertools.combinations(regions, 2)):
    data = [machine_region_dict[region_a].values, machine_region_dict[region_b].values]
    try:
        stat, p, dof, expected = chi2_contingency(data)
        #print(region_a, "vs ", region_b, "has p:",p)
        #set the machine_df data
        machine_df[region_a][region_b] = p
    except:
        print(region_a, region_b, "were too small")

Middle East Central Asia were too small
Middle East Oceania were too small
Oceania Central Asia were too small


In [73]:
machine_df

,Europe,Middle East,North America,Oceania,South East Asia,Latin America,Africa,Central Asia
Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Middle East,0.382948,NaN,NaN,NaN,NaN,NaN,NaN,NaN
North America,0.0,0.000062,NaN,NaN,NaN,NaN,NaN,NaN
Oceania,0.000081,NaN,0.779968,NaN,NaN,NaN,NaN,NaN
South East Asia,0.216581,0.056408,0.000238,0.002435,NaN,NaN,NaN,NaN
Latin America,0.000044,0.004362,0.106843,0.449329,0.021378,NaN,NaN,NaN
Africa,0.149486,0.054319,0.224736,0.332871,0.160286,0.806063,NaN,NaN
Central Asia,0.103675,NaN,0.449126,NaN,0.043402,0.759113,0.688436,NaN


In [80]:
machine_region_values = [[value] for value in machine_region_dict.values()]
type(machine_region_values)

list

In [82]:
from scipy import stats
stats.kruskal(*machine_region_values)

KruskalResult(statistic=array([7., 7., 7.]), pvalue=array([0.42887986, 0.42887986, 0.42887986]))

In [97]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom_test.html

# Check if the expected value of Chainsaws applications in Europe is greater than expected with random distribution (or equal parts )
for region in regions:
    print(region, "p=",stats.binom_test(machine_region_dict[region]["Chainsaw"], n = len(chi_test_data.loc[chi_test_data["Region"]=="Europe"]), p=0.125, alternative="greater"))

Europe p= 1.0712642393842817e-17
Middle East p= 0.22372283295741396
North America p= 0.9987479730822683
Oceania p= 0.9999999421337278
South East Asia p= 0.8968704041429194
Latin America p= 0.586604848350726
Africa p= 0.9999993619776829
Central Asia p= 0.9999999421337278
